<a href="https://colab.research.google.com/github/venu72561-chinnam/AI-ML-project/blob/main/Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers torch datasets

In [2]:
import pandas as pd
from datasets import Dataset

In [6]:
df=pd.read_csv('/content/offerings - Reviews_Preprocessed (1).csv')

In [7]:

dataset = Dataset.from_pandas(df)

In [8]:
# Split into train and validation sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [9]:
from transformers import GPT2Tokenizer

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [11]:
from transformers import GPT2Tokenizer
def tokenize_function(examples):
    return tokenizer(examples["preprocessed_reviews"], truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [12]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable wandb if you don't want it

In [14]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset

In [15]:
# Model and training setup
model = GPT2LMHeadModel.from_pretrained('gpt2')
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-hotel-reviews",
    run_name="gpt2-hotel-reviews-exp1",  # Descriptive run name
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=500,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=150, training_loss=6.21602294921875, metrics={'train_runtime': 36.3201, 'train_samples_per_second': 16.52, 'train_steps_per_second': 4.13, 'total_flos': 46022100480000.0, 'train_loss': 6.21602294921875, 'epoch': 3.0})

In [17]:
model.save_pretrained("./gpt2-hotel-reviews-final")
tokenizer.save_pretrained("./gpt2-hotel-reviews-final")

('./gpt2-hotel-reviews-final/tokenizer_config.json',
 './gpt2-hotel-reviews-final/special_tokens_map.json',
 './gpt2-hotel-reviews-final/vocab.json',
 './gpt2-hotel-reviews-final/merges.txt',
 './gpt2-hotel-reviews-final/added_tokens.json')